# Data Models and Query Languages

Data models have a critical impact on how we think about the problem we are solving and how software is written. 

Most applications are built by layering one data model on top of another. For each layer, the key question is how to represent data in next-lower layer. Example:
 * Application developers look at real world objects and represent/model them in terms of programmable objects or data structures and APIs that manipulate those data structures
 * For storing the data structures, we represent them in terms of general-purpose data model, like JSON or XML documents/tables/relational db/graph model
 * Engineers who built the db software decide how to represent JSON/XML/relational/graph data in terms of bytesin memory, disk or network
 * On lower levels, hardware engineers have figured out how to represent bytes in terms of electrical currents,pulses of light, magentic fields/etc

Each layer hides the complexity of layers below it by providing a clean data model

## Comparison between data models

### Relational Model vs Document Model

**SQL**

* SQL is based on relational model proposed by Edgar Codd in 1970
* Data is organised into relations (tables), where each relation in an unordered collection of tuples (rows)
* Although SQL was introduced for business data processing, it generalised well across newer use cases (most websites are still powered by SQL)


**NoSQL**

Driving forces behind adoption of NoSQL:
 * Need for greater scalability than relational databases for very large datasets or very high write throughput
 * Preference for free and open source software over commercial products
 * Specialised query operations that are not well supported by the relational models
 * Desire for more dynamic and expressive data model

Different applications have different requirements, and the best choice of technology for one use case may be different for other

Document and Graph DBs are two main types of NoSQL datastores (document dbs target self-contained documents where as graphs target use cases where anything is potentially related to everything)

**Object-Relational Mismatch**
 * Most application development is done in Object Oriented Programming
 * Translation layer is required between the objects and the relational model
 * This is sometimes called as impedance mismatch
 * Object-relational mapping (ORM) frameworks like ActiveRecord and Hibernate reduce number of boilerplate code, but cant hide differences between the two models
 * JSON representation has better locality than multi-table schema. Example: In case of Linkedin profile, we can store all dependencies in a single place than having multiple tables. This simplifies the queries and avoids the messy multi way joins across multiple tables.

**Many-to-One and Many-to-many relationships**
 * Normalizing data in relational dbs is essential for avoiding duplication of data
 * Unfortunately, normalizing data requires many-to-one relationships (many people live in one region) which does not fit well with the document model
 * In relational dbs, joins are common and hence ids are commonly used for this purpose
 * However, in document dbs, joins are not required for one-to-many tree structures and hence support for joins is often weak
 * If the underlying db itself does not support joins, you need to emulate joins in application code by making multiple queries to the db
 * Even if earlier version of application fits well in join free document model, eventual versions of application could require joins as data tends to become more interconnected as features are added to applications
 * Document models tend to work well for one-to-many relationships, but many-to-many relationships are difficult as joins are crucial for them
 * Both relational and document models use a unique id for many-to-one and many-to-many relationships. This id is called foreign key in relational model and document reference in document model

**Relational Versus Document Databases**
 * If the application has document-like structure (tree of one-to-many relationships, where entire tree is loaded at once), then document model can be preferred. The relational technique of shredding where we split a document like structure into multiple tables can lead to cumbersome schemas and unneccarily complicated application code
 * Document model has limitations - cannot refer directly to a nested item within a document, but need a path like query. This is mostly an issue only when documents are deeply nested. Poor join support for document models may be an issue depending on application at hand
 * If application uses many-to-many relationships, document model is less appealing
 * For highly connected data, relational model is acceptable and graph models are ideal
 * Document databases and JSON support in relational dbs do not enforce schema on data. 
 * This can be referred to as schema-on-read - which means schema is implicit and interpreted once data is read (opposite is schema-on-write which is the traditional way on explicitly conforming any written data to the schema). Schema-on-read is similar to dynamic (runtime) type checking in programming languages whereas schema-on-write is similar to static (compiler-time) type checking
 * Data locality - a document is often stored as a single continous string, encoded as JSON/XML/binary variants like BSON (MongoDB)
 * If application often needs to access the entire document, there is performance advantage to storage locality
 * On the other hand, if data is stored in multiple tables, multiple index lookups are required to retrieve which may need more disk seeks and time
 * Locality advantage only applies if you need large parts of document at same time. If we only need small portion of document, it is wasteful to load entire document. Therefore, it is better to keep documents fairly small and avoid writes that increase size of document
 * The idea of locality(grouping related data together) is not limited to document model. Google's Spanner db offers locality in relational model, Oracle allows same using multi-table index cluster tables. The column-family concept in Bigtable data model (Cassandra and HBase) has similar purpose of managing locality

**Convergence of document and relational databases**
 * Most relational dbs (other than MySQL) have supported XML. This includes functions to make local modifications to XML documents and ability to index and query inside XML documents. This allows applications to use data models similar to document db
 * PostgreSQL, MYSQL and IBM DB2 now have similar support for JSON documents
 * Similarly on document db side, RethinkDB supports relational-like joins
 * Relational and document dbs are becoming more similar over time

**Query Languages for Data**
 * SQL is a declarative query language, as opposed to being imperative
 * This means that in SQL, we only need to mention the conditions to be met, but now how to do the operation (order is decided by query optimiser)
 * Declarative languages often lend themselves to parallel execution where imperative are harder to parallel since the order of execution is specified/restricted
 * Declarative query languages are not limited to dbs, but also used in web browsers (CSS/XSL are declarative where as Javascript is imperative)

**MapReduce Querying**
 * MapReduce is a programming model for processing large amount of data in bulk across machines
 * Limited form of MapReduce is supported by some NoSQL datastores like MongoDB and CounchDB as a mechanism for read-only queries across documents
 * MapReduce is neither declarative nor imperative, but somewhere in between where logic of query is expressed via code snippets. 
 * MongoDB specific extension to MapReduce allows filtering via declarative language
 * Map and reduce functions are restricted in the sense that they must be "pure" functions and can only act on data passed to them
 * MapReduce is a fairly low-level programming model and higher level query languages like SQL can be implemented as a pipeline of MapReduce operations
 * There is nothing in SQL that restricts it to running on single machine and MapReduce doesn't have monopoly on distributed query execution
 * One usability problem with MapReduce is that we need to write two co-ordinated JS functions, which is harder than writing a single query. Also, a declarative language would offer more opportunity for query optimiser to improve performance. Therefore, MongoDB added support for declarative query language called 'aggregation pipeline'

**Graph-Like Data Models**
 * If application has mostly one-to-many relationships (tree-structured data) or no relationships, document model is appropriate
 * If application has a few many-to-many relationships, relational model can handle them
 * However, if connections with data are complex, it becomes more natural to model data as a graph
 * There are different but related ways of structuring and query data: Property graph model (implemented by Neo4j, Titan, InfiniteGraph) and the triple-store model (implemented by Datomic, AllegroGraph)
 * Cypher, SPARQL and Datalog are some of the declarative query languages for graphs. Gremlin is a imperative query language for graph

**Property Graphs**
 * Each vertex consists of a unique id, set of outgoing edges, incoming edges and a collection of properties (key-value pairs)
 * Each edge consists of a unique id, tail vertex (vertex at which edge starts), head vertex (vertex where edge ends), label to describe relationship between two vertices and a collection of properties (key-value pairs)
 * We can think of graph store as consisting of two relational tables, one for verticles and other for edges. However, this requires substantially more queries as compared to directly with graphs
 * Cypher is a declarative query language for property graphs created for Neo4j db graph db

**Triple-Stores and SPARQL**
 * Idea of triple stores is same as property graphs, however with a different terminology
 * In triple store, all info is stored in form of three-part statements: (subject, predicate, object). Example: (Jim, likes, Bananas)
 * Subject of a triple is equivalent to a vertex in graph. Object is either a value (ex: lucy, age, 33 - predicate and object are key/value of property on subject vertex) or another vertex (predicate is an edge)
 * The semantic web is completely independent of triple-stores, although many people find these two to be closely linked
 * The Resource Description Framework (RDF) was intended as a mechanism for different websites to publish data in consistent format - which would allow data from multiple websites to be combined into a web of data - internet wide database of everything 
 * Although semantic web itself did not gain much traction, the RDF triple store can be used by applications (without publishing anything on semantic web)

**RDF Data Model**
 * Turtle is a concise language to write RDF triples. RDF can also be written as XML, but is more verbose. Tools like Apache Jena can automatically convert between different RDF formats
 * Since RDF was designed for internet-wide data exchange, it has a few quirks: the subject, predicate and object of triple are often URIs. For example, a predicate might be a URI to allow combining your data with someone else's data with an externally defined predicate

**SPARQL query language**
 * SPARQL is a query language for triple-stores using RDF data model (full form is SPARQL Protocol and RDF Query Language, pronounced Sparkle)
 * SPARQL predates Cypher and looks similar as Cypher has borrowed pattern matching from SPARQL